In [1]:
import boto3
import json
import datetime
from IPython.display import display, JSON

bedrock_agent = boto3.client("bedrock-agent")
bedrock_runtime = boto3.client("bedrock-runtime")
bedrock_agent_runtime = boto3.client("bedrock-agent-runtime")

model_id = "amazon.nova-micro-v1:0"

In [2]:
prompt_name = f"job-description-{datetime.datetime.now().strftime('%Y%m%d%H%M%S')}"

template_text = '''
You are an HR assistant.

Write a professional, inclusive job description using the following inputs:

Job title: {{job_title}}
Responsibilities: {{responsibilities}}
Requirements: {{requirements}}
Location: {{location}}
Work type: {{work_type}}

- Start with a clear summary
- Use concise, inclusive language
- Keep it under 250 words
'''
response = bedrock_agent.create_prompt(
    name=prompt_name,
    description="Generates inclusive job descriptions from structured inputs",
    defaultVariant="v1",
    variants=[
        {
            "name": "v1",
            "modelId": model_id,
            "templateType": "TEXT",
            "templateConfiguration": {
                "text": {
                    "inputVariables": [
                        {"name": "job_title"},
                        {"name": "responsibilities"},
                        {"name": "requirements"},
                        {"name": "location"},
                        {"name": "work_type"}
                    ],
                    "text": template_text
                }
            },
            "inferenceConfiguration": {
                "text": {
                    "maxTokens": 500,
                    "temperature": 0.7,
                    "topP": 0.9,
                    "stopSequences": []
                }
            }
        }
    ]
)
print("\n==================== Response Object ====================\n")
display(JSON(response))

print("\n==================== Prompt ARN ====================\n")
print(response['arn'])


==================== Response Object ====================



<IPython.core.display.JSON object>


==================== Prompt ARN ====================

arn:aws:bedrock:us-east-1:120106008631:prompt/D8UQBG45AQ


In [3]:
# Create a new prompt version
response = bedrock_agent.create_prompt_version(
    description='Initial prompt for creating job description documents.',
    promptIdentifier="arn:aws:bedrock:us-east-1:120106008631:prompt/D8UQBG45AQ"
)

print("\n==================== PROMPT VERSION ARN ====================\n")
print(response["arn"])


==================== PROMPT VERSION ARN ====================

arn:aws:bedrock:us-east-1:120106008631:prompt/D8UQBG45AQ:1


In [4]:
# Invoke the model
prompt_arn_with_version = "arn:aws:bedrock:us-east-1:120106008631:prompt/D8UQBG45AQ:1"

response = bedrock_runtime.converse(
    modelId=prompt_arn_with_version,
    promptVariables={
        "job_title": {"text": "UX Designer"},
        "responsibilities": {"text": "Design user interfaces, run usability testing, collaborate with product teams"},
        "requirements": {"text": "3+ years experience, Figma, HTML/CSS knowledge, communication skills"},
        "location": {"text": "New York or remote"},
        "work_type": {"text": "Full-time"}
    }
)

print("\n==================== Response Text ====================\n")
print(response['output']['message']['content'][0]['text'])


==================== Response Text ====================

**Job Title: UX Designer**

**Summary:**
We are seeking a talented and inclusive UX Designer to join our dynamic team in New York or remotely. As a UX Designer, you will play a vital role in creating intuitive, user-friendly interfaces and conducting usability testing to enhance our products. Collaboration with cross-functional product teams is essential to ensure seamless user experiences.

**Responsibilities:**
- Design compelling user interfaces that meet user needs and business goals.
- Conduct and analyze usability testing to identify and resolve user experience issues.
- Collaborate closely with product, engineering, and design teams to deliver high-quality user experiences.
- Utilize Figma for prototyping and design systems.
- Write clean, maintainable HTML/CSS to implement design specifications.

**Requirements:**
- 3+ years of experience in UX design.
- Proficiency in Figma for design and prototyping.
- Strong knowledge

In [5]:
# Optimize prompt
def handle_response_stream(response):
    try:
        event_stream = response['optimizedPrompt']
        for event in event_stream:
            if 'optimizedPromptEvent' in event:
                print("\n==================== OPTIMIZED PROMPT ====================\n")
                print(event['optimizedPromptEvent']['optimizedPrompt']['textPrompt']['text'])
    except Exception as e:
        raise e


prompt_input = {
    "textPrompt": {
        "text": template_text
    }
}

response = bedrock_agent_runtime.optimize_prompt(
            input=prompt_input,
            targetModelId=model_id
        )

print("\n==================== ORIGINAL PROMPT ====================\n")
print(template_text)
handle_response_stream(response)


==================== ORIGINAL PROMPT ====================


You are an HR assistant.

Write a professional, inclusive job description using the following inputs:

Job title: {{job_title}}
Responsibilities: {{responsibilities}}
Requirements: {{requirements}}
Location: {{location}}
Work type: {{work_type}}

- Start with a clear summary
- Use concise, inclusive language
- Keep it under 250 words


==================== OPTIMIZED PROMPT ====================

"## Task\nWrite a professional and inclusive job description for the given role.\n\n## Instructions\n1. Review the provided inputs:\n### Job Title\n{{job_title}}\n\n### Responsibilities\n{{responsibilities}}\n\n### Requirements\n{{requirements}}\n\n### Location\n{{location}}\n\n### Work Type\n{{work_type}}\n\n2. Follow these guidelines to create the job description:\n\n### Job Description Structure\n- Summary: [Provide a clear and concise overview of the role, highlighting the key responsibilities and requirements]\n\n- Responsibilitie

In [6]:
# Create a new prompt
prompt_identifier = "arn:aws:bedrock:us-east-1:120106008631:prompt/D8UQBG45AQ"

existing_prompt = bedrock_agent.get_prompt(promptIdentifier=prompt_identifier)
print("\n==================== ORIGINAL PROMPT ====================\n")
print(existing_prompt["variants"][0]["templateConfiguration"]["text"]["text"])


updated_variants = existing_prompt["variants"]
for variant in updated_variants:
    if variant["templateType"] == "TEXT":
        variant["templateConfiguration"]["text"]["text"] = "## Task\nWrite a professional and inclusive job description for the given role.\n\n## Instructions\n1. Review the provided inputs:\n### Job Title\n{{job_title}}\n\n### Responsibilities\n{{responsibilities}}\n\n### Requirements\n{{requirements}}\n\n### Location\n{{location}}\n\n### Work Type\n{{work_type}}\n\n2. Follow these guidelines to create the job description:\n\n### Job Description Structure\n- Summary: [Provide a clear and concise overview of the role, highlighting the key responsibilities and requirements]\n\n- Responsibilities:\n    - [List the main responsibilities using bullet points, based on the provided {{responsibilities}}]\n\n- Requirements:\n    - [List the essential requirements using bullet points, based on the provided {{requirements}}]\n\n- Additional Details:\n    - Location: {{location}}\n    - Work Type: {{work_type}}\n\n3. Ensure the job description:\n    - Uses concise, inclusive, and professional language\n    - Avoids any discriminatory or biased language\n    - Stays within the 250-word limit\n\n4. Provide the job description immediately without any preamble or additional information."


response = bedrock_agent.update_prompt(
    promptIdentifier=prompt_identifier,
    name=existing_prompt["name"],
    description=existing_prompt["description"],
    defaultVariant=existing_prompt["defaultVariant"],
    variants=updated_variants
)

print("\n==================== UPDATED PROMPT ====================\n")

updated_prompt = bedrock_agent.get_prompt(promptIdentifier=prompt_identifier)
print(updated_prompt["variants"][0]["templateConfiguration"]["text"]["text"])


==================== ORIGINAL PROMPT ====================


You are an HR assistant.

Write a professional, inclusive job description using the following inputs:

Job title: {{job_title}}
Responsibilities: {{responsibilities}}
Requirements: {{requirements}}
Location: {{location}}
Work type: {{work_type}}

- Start with a clear summary
- Use concise, inclusive language
- Keep it under 250 words


==================== UPDATED PROMPT ====================

## Task
Write a professional and inclusive job description for the given role.

## Instructions
1. Review the provided inputs:
### Job Title
{{job_title}}

### Responsibilities
{{responsibilities}}

### Requirements
{{requirements}}

### Location
{{location}}

### Work Type
{{work_type}}

2. Follow these guidelines to create the job description:

### Job Description Structure
- Summary: [Provide a clear and concise overview of the role, highlighting the key responsibilities and requirements]

- Responsibilities:
    - [List the main resp

In [7]:
response = bedrock_agent.create_prompt_version(
    description='Optimized prompt for creating job description documents.',
    promptIdentifier=prompt_identifier
)

print("\n==================== PROMPT VERSION ARN ====================\n")
print(response["arn"])


==================== PROMPT VERSION ARN ====================

arn:aws:bedrock:us-east-1:120106008631:prompt/D8UQBG45AQ:2


#### Compare Versions

In [8]:
prompt_arn = "arn:aws:bedrock:us-east-1:120106008631:prompt/D8UQBG45AQ:1"

response = bedrock_runtime.converse(
    modelId=prompt_arn,
    promptVariables={
        "job_title": {"text": "UX Designer"},
        "responsibilities": {"text": "Design user interfaces, run usability testing, collaborate with product teams"},
        "requirements": {"text": "3+ years experience, Figma, HTML/CSS knowledge, communication skills"},
        "location": {"text": "New York or remote"},
        "work_type": {"text": "Full-time"}
    }
)

print("\\n==================== Response Text ====================\\n")
print(response['output']['message']['content'][0]['text'])

\n==================== Response Text ====================\n
**Job Title: UX Designer**

**Summary:**
We are seeking a talented and inclusive UX Designer to join our dynamic team in New York or remotely. As a UX Designer, you will be responsible for designing intuitive user interfaces, conducting usability testing, and collaborating closely with our product teams to enhance user experience. Your expertise in Figma, HTML/CSS, and strong communication skills will be pivotal in driving our projects forward.

**Responsibilities:**
- Design compelling and user-friendly interfaces.
- Conduct thorough usability testing to refine and improve user experiences.
- Collaborate effectively with cross-functional product teams to deliver exceptional user experiences.
- Utilize Figma for prototyping and design.
- Apply HTML/CSS knowledge to implement design specifications.

**Requirements:**
- 3+ years of experience in UX design.
- Proficiency in Figma.
- Strong understanding of HTML/CSS.
- Excellent c

In [9]:
prompt_arn = "arn:aws:bedrock:us-east-1:120106008631:prompt/D8UQBG45AQ:2"

response = bedrock_runtime.converse(
    modelId=prompt_arn,
    promptVariables={
        "job_title": {"text": "UX Designer"},
        "responsibilities": {"text": "Design user interfaces, run usability testing, collaborate with product teams"},
        "requirements": {"text": "3+ years experience, Figma, HTML/CSS knowledge, communication skills"},
        "location": {"text": "New York or remote"},
        "work_type": {"text": "Full-time"}
    }
)

print("\\n==================== Response Text ====================\\n")
print(response['output']['message']['content'][0]['text'])

\n==================== Response Text ====================\n
**Job Title: UX Designer**

**Summary:**
We are looking for a talented and inclusive UX Designer to join our team. The ideal candidate will design user interfaces, conduct usability testing, and collaborate with product teams to enhance user experience. This full-time position is available in New York or as a remote opportunity.

**Responsibilities:**
- Design intuitive and user-friendly interfaces
- Conduct usability testing to gather and analyze user feedback
- Collaborate with cross-functional product teams to implement design solutions

**Requirements:**
- 3+ years of experience in UX design
- Proficiency in Figma
- Knowledge of HTML/CSS
- Strong communication skills

**Additional Details:**
- Location: New York or remote
- Work Type: Full-time


In [10]:
# Cleanup resources
response = bedrock_agent.list_prompts()

for prompt in response['promptSummaries']:
    prompt_id = prompt['id']  # The correct key is 'id', not 'promptId'
    print(f"Deleting prompt: {prompt['name']} (ID: {prompt_id})")
    bedrock_agent.delete_prompt(promptIdentifier=prompt_id)

print("All prompts deleted successfully")

Deleting prompt: job-description-20250717110055 (ID: D8UQBG45AQ)
All prompts deleted successfully
